## inport package that need to use

In [2]:
#!pip install mediawikiapi
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from mediawikiapi import MediaWikiAPI

## set page wikipedi that need to scrape from Assignment

In [3]:
PageWithTables = 'List_of_postal_codes_of_Canada:_M'
# load page 
mediawikiapi = MediaWikiAPI()
test_page = mediawikiapi.page(PageWithTables)


## Functions to get data table then convert to pandas data frame

In [4]:
def wikitable_to_dataframe(table):
    """
    Exports a Wikipedia table parsed by BeautifulSoup. Deals with spanning: 
    multirow and multicolumn should format as expected. 
    """ 
    rows=table.findAll("tr")
    nrows=len(rows)
    ncols=max([len(r.findAll(['th','td'])) for r in rows])

    # preallocate table structure
    # (this is required because we need to move forward in the table
    # structure once we've found a row span)
    data=[]
    for i in range(nrows):
        rowD=[]
        for j in range(ncols):
            rowD.append('')
        data.append(rowD)

    # fill the table with data:
    # move across cells and use span to fill extra cells
    for i,row in enumerate(rows):    
        cells = row.findAll(["td","th"])
        for j,cell in enumerate(cells):        
            cspan=int(cell.get('colspan',1))
            rspan=int(cell.get('rowspan',1))
            l = 0
            for k in range(rspan):
                # Shifts to the first empty cell of this row
                # Avoid replacing previously insterted content
                while data[i+k][j+l]:
                    l+=1
                for m in range(cspan):
                    data[i+k][j+l+m]+=cell.text.strip("\n")

    return pd.DataFrame(data=data[1:],columns=data[0])

## use BeautifulSoup to transfrom html from wikipedia then find table that we interesting and send table to function

In [5]:
# scrape the HTML with BeautifulSoup to find tables
soup = BeautifulSoup(test_page.html(), 'html.parser')
tables = soup.findAll("table", { "class" : "wikitable" })

# select target table and apply custom function to export it to pandas
target_table = tables[0]
df_test = wikitable_to_dataframe(target_table)

## Clean Not assinged , Groupby then join data via Assignmet condition <br>
### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.<br>
### These two rows will be combined into one row with the neighborhoods.<br>
### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.<br>

#### only Postcode M7A  Neighbourhood is Not assigned so change it to same Borough that is Queen's Park


In [6]:
df = df_test[df_test['Borough']!='Not assigned']
df = df.reset_index(drop=True)
df['Neighbourhood'] = df.groupby(['Postcode'])['Neighbourhood'].transform(lambda x: ','.join(x))
df = df.drop_duplicates()
df = df.reset_index(drop=True)
#df = df[df['Neighbourhood']=='Not assigned']
df['Neighbourhood'] = df[['Neighbourhood','Borough']].apply(lambda x: x['Neighbourhood'] if x['Neighbourhood'] !='Not assigned' else x['Borough'], axis=1)
 
df.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


## show shape

In [7]:
print(df.shape)

(103, 3)


In [12]:
df_csv = pd.read_csv('http://cocl.us/Geospatial_data')

In [15]:
df_csv = df_csv.rename(index=str, columns={"Postal Code": "Postcode"})
df_csv.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
df = df.merge(df_csv,on='Postcode',how='left')

In [19]:
df.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
